<a href="https://colab.research.google.com/github/RoboticsLabURJC/2019-phd-sergio-paniego/blob/main/behavior_studio_networks/BS_classification_smaller_vgg_2_networks.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
# Mount Google Drive to access images dataset
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend as K

from keras.utils import plot_model, np_utils
import numpy as np
import glob
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import TensorBoard, CSVLogger, ModelCheckpoint

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

from time import time

def SmallerVGGNet(input_shape, num_classes):
    model = Sequential()

    model.add(Conv2D(32, (3, 3), padding='same', input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=-1))
    model.add(MaxPooling2D(pool_size=(3, 3)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=-1))
    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=-1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=-1))
    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=-1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=-1))
    model.add(Dropout(0.5))

    model.add(Dense(units=num_classes, activation='sigmoid'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

def get_images(list_images, type_image):
    # Read the images
    array_imgs = []
    for name in list_images:
        img = cv2.imread(name)
        if type_image == 'cropped':
            img = img[240:480, 0:640]
        img = cv2.resize(img, (int(img.shape[1] / 4), int(img.shape[0] / 4)))
        array_imgs.append(img)

    return array_imgs

def parse_json_7_classes_w(data):
    # Process json 7 classes for w
    array_class = []
    array_w = []
    data_parse = data.split('"class2": ')[1:]
    for d in data_parse:
        classification = d.split(', "class3":')[0]
        d_parse = d.split(', "w": ')[1]
        w = float(d_parse.split(', "v":')[0])
        array_class.append(classification)
        array_w.append(w)

    return array_class, array_w

def parse_json_4_classes_v(data):
    # Process json 54 classes for v
    array_class = []
    array_w = []
    data_parse = data.split('"class3": ')[1:]
    for d in data_parse:
        classification = d.split(', "w":')[0]
        d_parse = d.split(', "w": ')[1]
        w = float(d_parse.split(', "v":')[0])
        array_class.append(classification)
        array_w.append(w)

    return array_class, array_w

def parse_json_5_classes_v(data):
    # Process json 5 classes for v
    array_class = []
    array_w = []
    data_parse = data.split('"class_v_5": ')[1:]
    for d in data_parse:
        classification = d.split(', "class_w_9":')[0]
        d_parse = d.split(', "w": ')[1]
        w = float(d_parse.split(', "v":')[0])
        array_class.append(classification)
        array_w.append(w)

    return array_class, array_w

def remove_values_aprox_zero(list_imgs, list_data, list_w):
    index = [i for i,x in enumerate(list_w) if np.isclose([x], [0.0], atol=0.08)[0] == True]
    for i in range(len(index)-1, 0, -1):
        list_data.pop(index[i])
        list_imgs.pop(index[i])
    return list_imgs, list_data


def parse_json(data, num_classes, name_variable):
    if num_classes == 7 and name_variable == 'w':
        array_class, array_w = parse_json_7_classes_w(data)
    elif num_classes == 4 and name_variable == 'v':
        array_class, array_w = parse_json_4_classes_v(data)
    elif num_classes == 5 and name_variable == 'v':
        array_class, array_w = parse_json_5_classes_v(data)
    return array_class, array_w

def adapt_label_7_w(label):
    if label == '"radically_left"' or label == 'radically_left':
        label = 0
    elif label == '"moderately_left"' or label == 'moderately_left':
        label = 1
    elif label == '"slightly_left"' or label == 'slightly_left':
        label = 2
    elif label == '"slight"' or label == 'slight':
        label = 3
    elif label == '"slightly_right"' or label == 'slightly_right':
        label = 4
    elif label == '"moderately_right"' or label == 'moderately_right':
        label = 5
    elif label == '"radically_right"' or label == 'radically_right':
        label = 6
    return label


def adapt_label_4_v(label):
    if label == '"slow"' or label == 'slow':
        label = 0
    elif label == '"moderate"' or label == 'moderate':
        label = 1
    elif label == '"fast"' or label == 'fast':
        label = 2
    elif label == '"very_fast"' or label == 'very_fast':
        label = 3
    return label


def adapt_label_5_v(label):
    if label == '"slow"' or label == 'slow':
        label = 0
    elif label == '"moderate"' or label == 'moderate':
        label = 1
    elif label == '"fast"' or label == 'fast':
        label = 2
    elif label == '"very_fast"' or label == 'very_fast':
        label = 3
    elif label == '"negative"' or label == 'negative':
        label = 4
    return label


def adapt_labels(array_labels, num_classes, name_variable):
    for i in range(0, len(array_labels)):
        if name_variable == 'w':
            if num_classes == 7:
                array_labels[i] = adapt_label_7_w(array_labels[i])
        elif name_variable == 'v':
            if num_classes == 4:
                array_labels[i] = adapt_label_4_v(array_labels[i])
            elif num_classes == 5:
                array_labels[i] = adapt_label_5_v(array_labels[i])
    return array_labels



In [ ]:
!ls "/content/drive/My Drive"
!ls "/content/drive/My Drive/complete_dataset.zip"
!unzip "/content/drive/My Drive/curves_only.zip"
!unzip "/content/drive/My Drive/complete_dataset.zip"


import os 
#directory = os.listdir("/content/curves_only/Images")
#directory = os.listdir("/content/complete_dataset/Images")
#directory = os.listdir("/content/complete_dataset/Train_balanced_bbdd_v/Images")
#directory = os.listdir("/content/complete_dataset/Train_balanced_bbdd_w/Images")
directory = os.listdir("/content/complete_dataset/Train/Images")

import cv2
#img = cv2.imread("/content/curves_only/Images/4143.png")
#img = cv2.imread("/content/complete_dataset/Images/4143.png")
#img = cv2.imread("/content/complete_dataset/Train_balanced_bbdd_v/Images/4143.png")
#img = cv2.imread("/content/complete_dataset/Train_balanced_bbdd_w/Images/1363.png")
img = cv2.imread("/content/complete_dataset/Train/Images/4143.png")
height, width, channels = img.shape
# CHANGE num_classes
#num_classes = 7
#num_classes = 4
num_classes = 5


#list_images = glob.glob('/content/curves_only/Images/' + '*')
#list_images = glob.glob('/content/complete_dataset/Images/' + '*')
#list_images = glob.glob('/content/complete_dataset/Train_balanced_bbdd_v/Images/' + '*')
#list_images = glob.glob('/content/complete_dataset/Train_balanced_bbdd_w/Images/' + '*')
list_images = glob.glob('/content/complete_dataset/Train/Images/' + '*')


images = sorted(list_images, key=lambda x: int(x.split('/')[5].split('.png')[0]))

#file = open('/content/curves_only/data.json', 'r')
#file = open('/content/complete_dataset/data.json', 'r')
#file = open('/content/complete_dataset/Train_balanced_bbdd_v/train.json', 'r')
#file = open('/content/complete_dataset/Train_balanced_bbdd_w/train.json', 'r')
file = open('/content/complete_dataset/Train/train.json', 'r')

data = file.read()
file.close()
# CHANGE type_image
type_image = 'cropped'
#type_image='normal'

# CHANGE name_variable between v and w to generate the output for that specific
#name_variable='w'
name_variable='v'

print('----- LOADING DATASET ----')
x = get_images(images, type_image)
# Preprocess json
y, array_w = parse_json(data, num_classes, name_variable)


In [ ]:
print(img.shape)
if type_image == 'cropped':
    img_shape = (60, 160, 3)
else:
    img_shape = (120, 160, 3)
#img_shape = (120, 160, 3)
print(num_classes)
model = SmallerVGGNet(img_shape, num_classes)
print(model)

# We delete values close to zero
x_train, y_train = remove_values_aprox_zero(x, y, array_w)
#x_train = x
#y_train = y

y_train = adapt_labels(y_train, num_classes, name_variable)


# CHANGE type_net
# BALANCED DATASET OR NOT??
#type_net = 'balanced'
type_net = 'biased'



if type_net == 'balanced':
  X_train = x_train
  y_train = y_train
  X_t, X_validation, y_t, y_validation = train_test_split(x_train, y_train, test_size=0.20, random_state=42)
else:
  X_train, X_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size=0.20,
                                                                random_state=42)


# CHANGE batch_size
# CHANGE nb_epochs
batch_size = 64
# nb_epochs = 10
nb_epochs = 65

y_train = np_utils.to_categorical(y_train, num_classes)
y_validation = np_utils.to_categorical(y_validation, num_classes)

X_train = np.stack(X_train, axis=0)
y_train = np.stack(y_train, axis=0)
X_validation = np.stack(X_validation, axis=0)
y_validation = np.stack(y_validation, axis=0)

# CHANGE class_weight
if type_net == "biased" and num_classes == 7:
    nb_epochs = 55
    class_weight = {0: 4., 1: 2., 2: 2., 3: 1., 4:2., 5: 2., 6: 3.}
elif type_net == "biased" and num_classes == 5:
    class_weight = {0: 2., 1: 3., 2: 3., 3: 4., 4: 15}
elif type_net == "biased" and num_classes == 4:
    class_weight = {0: 2., 1: 3., 2: 3., 3: 4.}
else:
    class_weight = None

# Print layers
print(model.summary())

tensorboard = TensorBoard(log_dir="logs/{}".format(time()))

name_model='smaller_vgg'
#normal='normal'
#type_net = 'normal'

#  Train
model_history = model.fit(X_train, y_train, epochs=nb_epochs, batch_size=batch_size, verbose=2,
                          class_weight=class_weight, validation_data=(X_validation, y_validation),
                          callbacks=[tensorboard])


# Evaluate
score = model.evaluate(X_validation, y_validation, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# CHANGE Save model
#model_file = '/content/drive/My Drive/test_model_tf_keras_' + name_variable + '.h5'
#model_file = '/content/drive/My Drive/test_model_tf_keras_balanced_' + name_variable + '.h5'
model_file = '/content/drive/My Drive/test_model_tf_keras_cropped_' + type_net + '_' + name_variable + '.h5'
model.save(model_file)


print(model_file)

In [ ]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

%matplotlib inline

# Loss Curves
plt.figure(figsize=[8, 6])
plt.plot(model_history.history['loss'], 'r', linewidth=3.0)
plt.plot(model_history.history['val_loss'], 'b', linewidth=3.0)
plt.legend(['Training loss', 'Validation Loss'], fontsize=18)
plt.xlabel('Epochs ', fontsize=16)
plt.ylabel('Loss', fontsize=16)
plt.title('Loss Curves', fontsize=16)
plt.show()

# Accuracy Curves
plt.figure(figsize=[8, 6])
plt.plot(model_history.history['accuracy'], 'r', linewidth=3.0)
plt.plot(model_history.history['val_accuracy'], 'b', linewidth=3.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'], fontsize=18)
plt.xlabel('Epochs ', fontsize=16)
plt.ylabel('Accuracy', fontsize=16)
plt.title('Accuracy Curves', fontsize=16)
plt.show()